In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/var/folders/c4/kvf3th_j15lf5cq2pybvfgjm0000gn/T/ipykernel_42373/3590037574.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [4]:
len(embeddings.embed_query("Hello world"))

384

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [6]:
len(embeddings.embed_query("Hello world"))

768

In [17]:
from pinecone import Pinecone

os.environ['PINECONE_API_KEY'] = os.getenv("PINECONE_API_KEY")

In [18]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))